In [15]:

from pathlib import Path

from nna.fileUtils import list_files,save_to_csv

import numpy as np

In [16]:
import os
import sys

module_path = os.path.abspath(os.path.join('/home/enis/projects/similarSoundsApp/'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)
    

from nnSearch import loadOnDisk,buildOnDisk


/home/enis/projects/similarSoundsApp


#### Parameters

In [ ]:
indexIDname = "photoEmbeddings_V2_hamming"

outputFolder = Path("/scratch/enis/data/nna/realMerged/")
metadataCSV = outputFolder / (indexIDname+".csv")
fn = outputFolder / (indexIDname+".ann")

# vector size
f=128
  # Length of item vector that will be indexed
distance='hamming'
#prepares annoy to build the index in the specified file instead of RAM 
#(execute before adding items, no need to save after build)


### 1) Load embeddings to memory
#### 1.0) find images so we can just load embeddings from these locations


In [6]:
## Load datasets
realFolderPath="/scratch/enis/data/nna/real/"


# Look where the images are so we index only these locations
pathDatasets = "/home/enis/projects/similarSoundsApp/resources/TimeLapse_Databases/"
datasetList=[[m for m in i.glob("*") if ".ddb" in str(m) ][0] for i in (Path(pathDatasets).glob("*"))]
# locations with only images
labeledImgLocations = [m.parent.stem for m in datasetList]
# all existing locations
region_locId=[[m for m in i.glob("*") if m.is_dir() ] for i in (Path(realFolderPath).glob("*")) if i.is_dir()]

#keep locations only with images
allLocations=[]
for m in region_locId:
    for n in m:
        if n.stem in labeledImgLocations:
            allLocations.append(n)



#### 1.1 find embeddings in the given locations 

In [ ]:

filenamePattern="*_embeddings*.npy"
all_embeddings=[]
for loc in allLocations:
    # find all files
    search_path=loc
    all_embeddings+=list_files(str(search_path),filename=filenamePattern)

print(len(all_embeddings))

In [29]:
#number of embeddings 11235 for photoEmbeddings_V1

### 2.1) save metadata, embedding file names and sizes as a csv file

In [17]:
embedSizes=[]
for i,embed_file in enumerate(all_embeddings):
    embed=np.load(embed_file)
    embedSizes.append(embed.shape[0])

metadata = list(zip(all_embeddings,embedSizes))
save_to_csv(metadataCSV,metadata)

#### 2.2) Create and save annoy index

In [34]:
p=Path(fn)
if p.exists():
    print(p,"EXISTS")
    t = loadOnDisk(fn,f,distance=distance)
else:
    t = buildOnDisk(fn,f,distance=distance,seed=42,treeN=16)

Current index: 0
Current index: 1000
Current index: 2000
Current index: 3000
Current index: 4000
Current index: 5000
Current index: 6000
Current index: 7000
Current index: 8000
Current index: 9000
Current index: 10000
Current index: 11000


### 3) create functions caches depending on the Index and Metadata

#### 3.1) npy2originalFile

In [ ]:
#NNA folder
nnaProjectDataFolder = Path("/home/enis/projects/nna/data/")

# name and folder of the cached function  file
npy2originalFileCacheFileName = "npy2originalFileCache_"+indexIDname+".pkl"
pklPath = nnaProjectDataFolder / "funcCache" / npy2originalFileCacheFileName


# parameters for the npy2originalFile
file_properties_df_FilePath = nnaProjectDataFolder / "allFields_dataV3.pkl"

embeddataPath="/scratch/enis/data/nna/real/"
outputPath=embeddataPath
inputPath="/tank/data/nna/real/"


In [ ]:
from nna.fileUtils import npy2originalFile


In [ ]:
# parameters ,inputPath,outputPath,file_properties_df

npy2originalFileCache={}
for i,meta in enumerate(metadata):
    orgFilePath = npy2originalFile(Path(meta[0]),inputPath,outputPath,
                                               file_properties_df,subDirectoryAddon=None,
                                               fileNameAddon=None,debug=0)
    npy2originalFileCache[meta[0]] = orgFilePath
    if i%1000==0:
        print(i)

In [ ]:
npy2originalFileCache["params"] = [None,inputPath,outputPath,str(file_properties_df_FilePath),None,None,0]

In [ ]:
import pickle


with open(pklPath, 'wb') as handle:
    pickle.dump(npy2originalFileCache, handle, protocol=pickle.HIGHEST_PROTOCOL)

    
with open(pklPath, 'rb') as handle:
    b = pickle.load(handle)

print (npy2originalFileCache == b)